In [12]:
import osgeo
srs = SpatialReference()


NameError: name 'SpatialReference' is not defined

In [5]:
osgeo.__version__

'3.7.0'

In [13]:
import os
from osgeo import gdal, ogr, osr
from sklearn.model_selection import train_test_split
import numpy as np

gdal.UseExceptions()

In [14]:
source = gdal.Open(
    "/vsicurl/https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/10/T/FK/2021/7/S2B_10TFK_20210713_0_L2A/SCL.tif"
)

scl = source.GetRasterBand(1)

In [19]:
def pixel_to_coords(source, x, y):
    """Returns global coordinates in EPSG:4326 from pixel x, y coords"""

    geo_transform = source.GetGeoTransform()
    print(geo_transform)
    
    x_min = geo_transform[0]
    x_size = geo_transform[1]
    y_min = geo_transform[3]
    y_size = geo_transform[5]
    px = x * x_size + x_min
    py = y * y_size + y_min

    srs = osr.SpatialReference()
    
    if int(osgeo.__version__[0]) >= 3:
        # GDAL 3 changes axis order: https://github.com/OSGeo/gdal/issues/1546
        srs.SetAxisMappingStrategy(osgeo.osr.OAMS_TRADITIONAL_GIS_ORDER)
    
    srs.ImportFromWkt(source.GetProjection())

    srs_4326 = srs.CloneGeogCS()
    ct = osr.CoordinateTransformation(srs, srs_4326)

    long, lat, _ = ct.TransformPoint(px, py)

    return long, lat

In [20]:
source.GetGeoTransform()

(600000.0, 20.0, 0.0, 4500000.0, 0.0, -20.0)

In [21]:
# os.environ["GDAL_DATA"] = "/opt/conda/envs/env_label/share/gdal"
pixel_to_coords(source, 100, 100)

(600000.0, 20.0, 0.0, 4500000.0, 0.0, -20.0)


(-121.79397555246852, 40.626541370770724)

In [ ]:
np.random.seed(42)
xy = np.random.randint(1, 5490, size=(500, 2))

In [ ]:
x_values = []
y_values = []

for pos in xy:
    x_values.append([*pixel_to_coords(source, pos[0], pos[1])])

    y_values.append(
        int(
            scl.ReadAsArray(
                xoff=int(pos[0]), yoff=int(pos[1]), win_xsize=1, win_ysize=1
            )[0][0]
        )
    )
# x_values

In [ ]:
# Get all pixels and SCL values
# x_values = []
# y_values = []
# for px in range(0, 5490, 200):
#     for py in range(0, 5490, 200):
#         x_values.append([*pixel_to_coords(source, px, py)])
#         y_values.append(int(scl.ReadAsArray(px, py, 1, 1)[0][0]))

In [ ]:
np.array(y_values).shape

In [ ]:
x_train, x_rem, y_train, y_rem = train_test_split(
    np.array(x_values), np.array(y_values), train_size=0.8
)

In [ ]:
x_valid, x_test, y_valid, y_test = train_test_split(
    np.array(x_rem), np.array(y_rem), test_size=0.5
)

In [ ]:
x_valid.shape, x_train.shape, x_test.shape

In [ ]:
y_valid.shape, y_train.shape, y_test.shape

In [ ]:
y_valid

In [ ]:
def to_geojson(t, x, y):
    """Converts the given x, y, and split dataset type (train, test, validate ) to a geojson file
    The geojson file is saved in the current directory with the name label-{t}.geojson
    """

    field_name = "class"
    field_type = ogr.OFTInteger

    # Create the output Driver
    out_driver = ogr.GetDriverByName("GeoJSON")

    geojson_filename = f"label-{t}.geojson"
    # Create the output GeoJSON
    out_datasource = out_driver.CreateDataSource(geojson_filename)
    out_layer = out_datasource.CreateLayer("labels", geom_type=ogr.wkbPolygon)
    id_field = ogr.FieldDefn(field_name, field_type)
    out_layer.CreateField(id_field)
    # Get the output Layer's Feature Definition
    feature_def = out_layer.GetLayerDefn()

    for index, v in enumerate(y):
        point = ogr.Geometry(ogr.wkbPoint)
        point.AddPoint(x[index][0], x[index][1])

        # create a new feature
        out_feature = ogr.Feature(feature_def)

        # Set new geometry
        out_feature.SetGeometry(point)

        out_feature.SetField(field_name, int(v))
        # Add new feature to output Layer
        out_layer.CreateFeature(out_feature)

        # dereference the feature
        out_feature = None

    # Save and close DataSources
    out_datasource = None

In [ ]:
to_geojson("train_0", x_train, y_train)
# to_geojson("test", x_test, y_test)
# to_geojson("validate", x_valid, y_valid)